In [ ]:
import torch
from torch import nn
import torch.nn.init as init

class ACA(nn.Module):
    expansion = 1
    def __init__(self, in_channel,stride, downsample=None,  ratio=4):
        super(ACA, self).__init__()
        ratio=4
        hide_channel = in_channel // ratio
        self.avg_pool = nn.AdaptiveAvgPool1d(1)  # 将2d改为1d
        self.conv1 = nn.Conv1d(in_channel, hide_channel, kernel_size=1, bias=False)  # 将2d改为1d
        self.softmax = nn.Softmax(dim=1)  # softmax的维度保持不变
        self.A0 = torch.eye(hide_channel).to('cuda' if torch.cuda.is_available() else 'cpu')
        self.A2 = nn.Parameter(torch.zeros(hide_channel, hide_channel), requires_grad=True)
        init.constant_(self.A2, 1e-6)
        self.conv2 = nn.Conv1d(1, 1, kernel_size=1, bias=False)  # 保持不变
        self.conv3 = nn.Conv1d(1, 1, kernel_size=1, bias=False)  # 保持不变
        self.relu = nn.ReLU(inplace=True)
        self.conv4 = nn.Conv1d(hide_channel, in_channel, kernel_size=1, bias=False)  # 将2d改为1d
        self.sigmoid = nn.Sigmoid()
        self.bn = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        

    def forward(self, x):
        # x: (B, C, L)
        y = self.avg_pool(x)  # (B, C, 1)
        y1 = self.conv1(y)  # (B, hide_channel, 1)
        y2 = self.conv1(y)
        y = y1 + y2
        y = torch.mean(torch.stack([y1, y2],2), 2)
        y = self.relu(self.bn(y))

        
        B, C, _ = y.size()
        y = y.transpose(1, 2)  # (B, 1, C/r) 
        # print(y.size())
        # A1 = self.softmax(self.conv2(y))  # ( B，1，C/r)
        

        
        A1 = self.softmax(y)
        # A1 = A1.unsqueeze(0)  # (C, B, 1)
        A = (self.A0 * A1) + self.A2  # (C, B)
        y = torch.matmul(y, A)  # (C, B) * (C, B) -> (C, B)
        y = self.relu(self.conv3(y))  # (B,1,C/r)
        # print(y.size())
        y = y.transpose(1, 2)  # (B，1，C/r) -> (B, C, 1)
        y = self.sigmoid(self.conv4(y))  # (B, C, 1)

        return x * y  # (B, C, L) * (B, C, 1) -> (B, C, L)

class Model(nn.Module):
    def __init__(self, configs, hparams):
        super(Model, self).__init__()

        filter_sizes = [5, 9, 11,3]
        self.conv4 = nn.Conv1d(configs.input_channels, configs.mid_channels, kernel_size=filter_sizes[3],
                               stride=configs.stride, bias=False, padding=(filter_sizes[3] // 2))
        self.conv1 = nn.Conv1d(configs.input_channels, configs.mid_channels, kernel_size=filter_sizes[0],
                               stride=configs.stride, bias=False, padding=(filter_sizes[0] // 2))
        self.conv2 = nn.Conv1d(configs.input_channels, configs.mid_channels, kernel_size=filter_sizes[1],
                               stride=configs.stride, bias=False, padding=(filter_sizes[1] // 2))
        self.conv3 = nn.Conv1d(configs.input_channels, configs.mid_channels, kernel_size=filter_sizes[2],
                               stride=configs.stride, bias=False, padding=(filter_sizes[2] // 2))
        self.bn = nn.BatchNorm1d(configs.mid_channels)
        self.relu = nn.ReLU()
        self.mp = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.do = nn.Dropout(configs.dropout)
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(configs.mid_channels, configs.mid_channels * 2, kernel_size=8, stride=1, bias=False, padding=4),
            nn.BatchNorm1d(configs.mid_channels * 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv1d(configs.mid_channels * 2, configs.final_out_channels, kernel_size=8, stride=1, bias=False,
                      padding=4),
            nn.BatchNorm1d(configs.final_out_channels),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2, padding=1),
        )
        
        self.inplanes = 128
        # self.cbam = self._make_layer(ACA, 128, 1)
        self.aca = ACA(128,1)

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=configs.trans_dim, nhead=configs.num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        
        self.aap = nn.AdaptiveAvgPool1d(1)
        self.clf = nn.Linear(hparams["feature_dim"], configs.num_classes)
    def forward(self, x_in):
        # print(x_in.shape)

        # Multi-scale Convolutions
        x1 = self.conv1(x_in)
        x2 = self.conv2(x_in)
        x3 = self.conv3(x_in)
        x4 = self.conv4(x_in)
        x_concat = torch.mean(torch.stack([x1, x2, x3,x4],2), 2)
        x_concat = self.do(self.mp(self.relu(self.bn(x_concat))))

        x = self.conv_block2(x_concat)
        x = self.conv_block3(x)

        # ACA module
        x = self.aca(x)
        
        # Bi-directional Transformer
        x1 = self.transformer_encoder(x)
        x2 = self.transformer_encoder(torch.flip(x,[2]))


        x = x1+x2

        x = self.aap(x)
        x_flat = x.reshape(x.shape[0], -1)
        x_out = self.clf(x_flat)
        return x_out



